<hr style="border:0.1px solid gray">

# <center style='color:Red'> COSC2753 - Machine Learning </center> 
## <center style='color:Red'> Assignment 2 - Classify Images of Colon Cancer </center>
### <center style='color:Red'> Task 3 - Classify extra data cell type </center>

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#-COSC2753---Machine-Learning-" data-toc-modified-id="-COSC2753---Machine-Learning--1"><center style="color:Red"> COSC2753 - Machine Learning </center></a></span><ul class="toc-item"><li><span><a href="#-Assignment-2---Classify-Images-of-Colon-Cancer-" data-toc-modified-id="-Assignment-2---Classify-Images-of-Colon-Cancer--1.1"><center style="color:Red"> Assignment 2 - Classify Images of Colon Cancer </center></a></span><ul class="toc-item"><li><span><a href="#-Task-3---Classify-extra-data-cell-type-" data-toc-modified-id="-Task-3---Classify-extra-data-cell-type--1.1.1"><center style="color:Red"> Task 3 - Classify extra data cell type </center></a></span></li></ul></li></ul></li><li><span><a href="#Main-Task" data-toc-modified-id="Main-Task-2">Main Task</a></span></li><li><span><a href="#1.-Import-libraries-and-setting-up-environment" data-toc-modified-id="1.-Import-libraries-and-setting-up-environment-3">1. Import libraries and setting up environment</a></span></li><li><span><a href="#2.-Import-Dataset" data-toc-modified-id="2.-Import-Dataset-4">2. Import Dataset</a></span></li><li><span><a href="#3.-Exploratory-Data-Analysis-(EDA)" data-toc-modified-id="3.-Exploratory-Data-Analysis-(EDA)-5">3. Exploratory Data Analysis (EDA)</a></span><ul class="toc-item"><li><span><a href="#3.1.-Process-the-data" data-toc-modified-id="3.1.-Process-the-data-5.1">3.1. Process the data</a></span></li><li><span><a href="#3.2.-Data-Analysis-and-Visualiztion" data-toc-modified-id="3.2.-Data-Analysis-and-Visualiztion-5.2">3.2. Data Analysis and Visualiztion</a></span></li><li><span><a href="#3.3.-Data-Preparation" data-toc-modified-id="3.3.-Data-Preparation-5.3">3.3. Data Preparation</a></span></li></ul></li><li><span><a href="#4.-Building-GAN-model" data-toc-modified-id="4.-Building-GAN-model-6">4. Building GAN model</a></span><ul class="toc-item"><li><span><a href="#4.1-Function-define" data-toc-modified-id="4.1-Function-define-6.1">4.1 Function define</a></span></li><li><span><a href="#------------------------" data-toc-modified-id="-------------------------6.2">------------------------</a></span><ul class="toc-item"><li><span><a href="#Image-functions" data-toc-modified-id="Image-functions-6.2.1">Image functions</a></span></li></ul></li><li><span><a href="#------------------------" data-toc-modified-id="-------------------------6.3">------------------------</a></span><ul class="toc-item"><li><span><a href="#Generate-samples" data-toc-modified-id="Generate-samples-6.3.1">Generate samples</a></span></li></ul></li><li><span><a href="#------------------------" data-toc-modified-id="-------------------------6.4">------------------------</a></span><ul class="toc-item"><li><span><a href="#Generator" data-toc-modified-id="Generator-6.4.1">Generator</a></span></li></ul></li><li><span><a href="#------------------------" data-toc-modified-id="-------------------------6.5">------------------------</a></span><ul class="toc-item"><li><span><a href="#Discriminator" data-toc-modified-id="Discriminator-6.5.1">Discriminator</a></span></li></ul></li><li><span><a href="#------------------------" data-toc-modified-id="-------------------------6.6">------------------------</a></span><ul class="toc-item"><li><span><a href="#GAN" data-toc-modified-id="GAN-6.6.1">GAN</a></span></li></ul></li><li><span><a href="#------------------------" data-toc-modified-id="-------------------------6.7">------------------------</a></span></li><li><span><a href="#4.2-Model-training" data-toc-modified-id="4.2-Model-training-6.8">4.2 Model training</a></span></li><li><span><a href="#4.3-Prediction" data-toc-modified-id="4.3-Prediction-6.9">4.3 Prediction</a></span></li></ul></li></ul></div>

**Group Number:** 8

**Student 1:** Ngo Quang Khai - s3836387 

**Student 2:** Nguyen Quoc Minh - s3758994

**Student 3:** Bui Thanh Huy - s3740934

**Student 4:** Tran Vinh Khang - s3855823

**Student 5:** Le Trong Hung - s3805504

**Lecturer:** Dr. Bao Nguyen

**Due Date:** May 14th, 2021

<hr style="border:0.1px solid gray">

# Main Task

- **Task**: Classify cell type on extra data

# 1. Import libraries and setting up environment

In [5]:
# !pip install tensorflow
# !pip install keras
# !pip install opencv-python
# !pip install bayesian-optimization 

In [6]:
# Import packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import sklearn as sk
import seaborn as sns
import cv2
import os
import tensorflow as tf

from numpy import expand_dims, zeros, ones, asarray
from numpy.random import randn, randint
from tqdm import tqdm
from sklearn.model_selection import train_test_split, cross_val_score
from imblearn.over_sampling import RandomOverSampler

from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam, SGD, RMSprop, Adadelta, Adagrad, Adamax, Nadam, Ftrl
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from tensorflow.keras import layers
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.regularizers import l2
from keras import backend as K
from math import floor
from sklearn.metrics import make_scorer, accuracy_score, classification_report

from sklearn.model_selection import StratifiedKFold,GridSearchCV

from PIL import Image
import warnings
warnings.filterwarnings('ignore')

LeakyReLU = LeakyReLU(alpha=0.1)

In [7]:
#It is important to check the version of these libraries to make sure that implement the latest method.
print('Tensorflow version:', tf.__version__)
print('CV2 version:', cv2.__version__)

Tensorflow version: 2.6.0
CV2 version: 4.5.5


# 2. Import Dataset

In [8]:
# Import data_labels_mainData.csv into a DataFrame
main_data = pd.read_csv('data_labels_mainData.csv')

# Import data_labels_extraData.csv into a DataFrame
extra_data = pd.read_csv('data_labels_extraData.csv')

# 3. Exploratory Data Analysis (EDA)

## 3.1. Process the data

In [9]:
#See the data frame of main_data
print("Dataframe of main_data: \n", main_data, "\n")

#See the data frame of extra_data
print("Dataframe of extra_data: \n", extra_data)

Dataframe of main_data: 
       InstanceID  patientID  ImageName cellTypeName  cellType  isCancerous
0          22405          1  22405.png   fibroblast         0            0
1          22406          1  22406.png   fibroblast         0            0
2          22407          1  22407.png   fibroblast         0            0
3          22408          1  22408.png   fibroblast         0            0
4          22409          1  22409.png   fibroblast         0            0
...          ...        ...        ...          ...       ...          ...
9891        1625         60   1625.png   epithelial         2            1
9892        1626         60   1626.png   epithelial         2            1
9893        1627         60   1627.png   epithelial         2            1
9894        1628         60   1628.png   epithelial         2            1
9895        1629         60   1629.png   epithelial         2            1

[9896 rows x 6 columns] 

Dataframe of extra_data: 
        InstanceID  p

**=>** Next, we will check if the data have any null values and datatype of each columns.

In [10]:
main_data.info()
main_data.isnull().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9896 entries, 0 to 9895
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   InstanceID    9896 non-null   int64 
 1   patientID     9896 non-null   int64 
 2   ImageName     9896 non-null   object
 3   cellTypeName  9896 non-null   object
 4   cellType      9896 non-null   int64 
 5   isCancerous   9896 non-null   int64 
dtypes: int64(4), object(2)
memory usage: 464.0+ KB


InstanceID      0
patientID       0
ImageName       0
cellTypeName    0
cellType        0
isCancerous     0
dtype: int64

In [11]:
extra_data.info()
extra_data.isnull().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10384 entries, 0 to 10383
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   InstanceID   10384 non-null  int64 
 1   patientID    10384 non-null  int64 
 2   ImageName    10384 non-null  object
 3   isCancerous  10384 non-null  int64 
dtypes: int64(3), object(1)
memory usage: 324.6+ KB


InstanceID     0
patientID      0
ImageName      0
isCancerous    0
dtype: int64

**=>** Since our data is no missing/null value and properly formatted, we will move on to analyzing data.

## 3.2. Data Analysis and Visualiztion

Based on the information provided by the biomedical company as well as the DataFrames in **section 3.1** above. We are aware of:
- The **main_data** contains both **cellType (0 to 3)** and **isCancerous (0 and 1)** columns. In there, **cellType** values 0 to 3 represent fibroblast, inflammatory, epithelial, and miscellaneous cell types respectively based on the **cellTypeName** column. In addition, **isCancerous** values 0 and 1 represent colon cells with non-Cancer and Cancer.


- The **extra_data** only contains **isCancerous (0 and 1)** column.

**=>** Therefore, we choose the **isCancerous** attribute as the target column because both **main_data** and **extra_data** have the column **isCancerous**.



- In **main_data**, only the attribute **cellType** continue to be our strongest candidate for to influence our model when it has high impact on our target attribute is **isCancerous**.


- Attributes such as **InstanceID** and **patientID** might create a lot of noise in our model. 


- There are 4 types of cells: 0, 1, 2, 3. Only cell type number 2 is cancerous while others are not cancerous.


- We expect the the model would predict class 0 more accurately than class 1

## 3.3. Data Preparation

In [12]:

traindata, validata = train_test_split(main_data[['ImageName','cellType']], 
                                              test_size=0.2, random_state=9, stratify = main_data[['cellType']])                                        
trainExtra, valiExtra = train_test_split(main_data[['ImageName']], 
                                              test_size=0.2, random_state=9, stratify = main_data[['cellType']])   
print('Training data shape task 1:', traindata.shape)
print('Validation data shape task 1:', validata.shape)
print("Sampled Dataset:\n",traindata['cellType'].value_counts())
print("Sampled Dataset:\n",)


Training data shape task 1: (7916, 2)
Validation data shape task 1: (1980, 2)
Sampled Dataset:
 2    3263
1    2034
0    1510
3    1109
Name: cellType, dtype: int64
Sampled Dataset:



# 4. Building GAN model

## 4.1 Function define

## ------------------------

### Image functions

These are the image related functions

In [13]:
#Create a a function to add image according to the name given from the list

def GetImage(directory):
    images=[]
    for name in tqdm(directory, desc="Adding images"):
        image = cv2.imread("patch_images/"+name)
        dim = (28,28)
        image = cv2.resize(image, dim)
        image = Image.fromarray(image,'RGB')
        images.append(np.array(image))
    result = np.array(images)
    print("\ngetImage COMPLETED!")
    return result



In [14]:
# Get images ready by loading and preprocessing.
def load_main_samples(traindata):
    trainX = GetImage(traindata['ImageName'])
    trainy = traindata['cellType']

    X = trainX.astype('float32')
    X = (X - 127.5) / 127.5  # scale from [0,255] to [-1,1] as we will be using tanh activation.
    return [X, trainy]


In [15]:
# Get images ready by loading and preprocessing.
def load_extra_samples(exTrain):
    trainX = GetImage(exTrain['ImageName'])
    X = trainX.astype('float32')
    X = (X - 127.5) / 127.5  # scale from [0,255] to [-1,1] as we will be using tanh activation.
    return X

In [16]:
#select subset of the dataset for supervised training
#Let us pick only 3324 samples to be used in supervised training.
#Also, we need to ensure we pick 10 samples per class to ensure a good balance
#of data between classes. 

def select_supervised_samples(dataset, n_samples=4400, n_classes=4):
    X, y = dataset
    X_list, y_list = list(), list()
    n_per_class = int(n_samples / n_classes) #Number of amples per class. 
    for i in range(n_classes):
        X_with_class = X[y == i] # get all images for this class
        ix = randint(0, len(X_with_class), n_per_class) # choose random images for each class
        [X_list.append(X_with_class[j]) for j in ix] # add to list
        [y_list.append(i) for j in ix]
    return asarray(X_list), asarray(y_list) #Returns a list of 2 numpy arrays corresponding to X and Y


## ------------------------

### Generate samples

This is the sample generated code

In [17]:
#Create a function to generate sample to fix the Imblance of the dataset
def GenerateSample(X,Y):
    ros = RandomOverSampler(random_state = 1)
    x, y = ros.fit_resample(X.values.reshape(-1,1), Y)
    x = x.flatten()
    return x,y

In [18]:
# Pick real samples from the dataset.
#Return both images and corresponding labels in addition to y=1 indicating that the images are real. 
#Remember that we will not use the labels for unsupervised, only used for supervised. 
def fetch_sup_samples(dataset, n_samples):
    images, labels = dataset
    ix = randint(0, images.shape[0], n_samples)
    X, labels = images[ix], labels[ix] #Select random images and corresponding labels 
    y = ones((n_samples, 1)) #Label all images as 1 as these are real images. (for the discriminator training)
    return [X, labels], y


In [19]:
# Pick real samples from the dataset.
#Return both images and corresponding labels in addition to y=1 indicating that the images are real.
#Remember that we will not use the labels for unsupervised, only used for supervised.
def fetch_unsup_samples(dataset, n_samples):
    images = dataset
    ix = randint(0, images.shape[0], n_samples)
    X = images[ix] #Select random images and corresponding labels
    y = ones((n_samples, 1)) #Label all images as 1 as these are real images. (for the discriminator training)
    return X, y


In [20]:
# Generate fake images using the generator and above latent points as input to it. 
#We do not care about labeles so the generator will not know anything about the labels. 
def generate_fake_samples(generator, latent_dim, n_samples):
	z_input = generate_latent_points(latent_dim, n_samples)
	fake_images = generator.predict(z_input)
	# create class labels
	y = zeros((n_samples, 1)) #Label all images as 0 as these are fake images. (for the discriminator training) 
	return fake_images, y


In [21]:
# generate latent points, to be used as inputs to the generator. 
def generate_latent_points(latent_dim, n_samples):
	z_input = randn(latent_dim * n_samples)
	z_input = z_input.reshape(n_samples, latent_dim) # reshape for input to the network
	return z_input

## ------------------------

### Generator

This is the generator model code

In [22]:

n_nodes = 256 * 7 *7
X = Sequential([
    layers.Dense(n_nodes, input_dim=100),
    layers.LeakyReLU(alpha=0.2),
    layers.Reshape((7, 7, 256)),
    
    layers.Conv2DTranspose(128, (3,3), strides=(2,2), padding='same'),#14x14x128
    layers.LeakyReLU(alpha=0.2),
    
    layers.Conv2DTranspose(64, (5,5), strides=(1,1), padding='same'), #14x14x64
    layers.LeakyReLU(alpha=0.2),
    # output
    layers.Conv2DTranspose(3, (3,3), strides=(2,2), activation='tanh',padding='same')
])

opt = Adam(lr=0.0005, beta_1=0.4, beta_2=0.444, decay=1.0e-6, amsgrad=True)
X.compile(optimizer=opt , loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), metrics=['accuracy'])
X.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 12544)             1266944   
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 12544)             0         
_________________________________________________________________
reshape (Reshape)            (None, 7, 7, 256)         0         
_________________________________________________________________
conv2d_transpose (Conv2DTran (None, 14, 14, 128)       295040    
_________________________________________________________________
leaky_re_lu_3 (LeakyReLU)    (None, 14, 14, 128)       0         
_________________________________________________________________
conv2d_transpose_1 (Conv2DTr (None, 14, 14, 64)        204864    
_________________________________________________________________
leaky_re_lu_4 (LeakyReLU)    (None, 14, 14, 64)        0

In [23]:
# define the standalone generator model
def define_generator(latent_dim):
    model = Sequential()
	# We will reshape input latent vector into 7x7 image as a starting point. 
    #So n_nodes for the Dense layer can be 128x8x8 so when we reshape the output 
    #it would be 8x8x128 and that can be slowly upscaled to 32x32 image for output.
    n_nodes = 256 * 7 *7  #8192 nodes
    
    model.add(layers.Dense(n_nodes, input_dim=latent_dim)) #Dense layer so we can work with 1D latent vector
    model.add( layers.LeakyReLU(alpha=0.2))
    model.add( layers.Reshape((7, 7, 256)))  #8x8x128 dataset from the latent vector.
    
    # upsample to 14x14
    model.add( layers.Conv2DTranspose(128, (3,3), strides=(1,1), padding='same')) #14x14x128
    model.add( layers.LeakyReLU(alpha=0.2))
    
    # upsample to 28x28
    model.add( layers.Conv2DTranspose(64, (5,5), strides=(2,2), padding='same')) #14x14x64
    model.add( layers.LeakyReLU(alpha=0.2))
    
    # generate
    model.add( layers.Conv2DTranspose(3, (3,3), strides=(2, 2), activation='tanh', padding='same')) #28x28x3
    assert model.output_shape == (None, 28, 28, 3)
    return model  #Model not compiled as it is not directly trained like the discriminator.
                    #Generator is trained via GAN combined model. 


test_gen = define_generator(100)
print(test_gen.summary())

## ------------------------

### Discriminator

In [24]:
# define the base discriminator model for sup and unsup discriminators
#This is the base discriminator that supervised and unsupervised are going to share weights from. 
#(I know that the above ssentence is bad english!!!)

def define_discriminator(in_shape=(28,28,3), n_classes=4):
    model = Sequential([
        layers.Conv2D(32, (3, 3),strides=(2,2), kernel_initializer='he_normal', padding='same'),
        layers.LeakyReLU(alpha=0.2),
        layers.Conv2D(32, (3, 3),strides=(2,2), kernel_initializer='he_normal', padding='same'),
        layers.LeakyReLU(alpha=0.2),
        layers.Dropout(0.2),
        
        layers.Conv2D(64, (3, 3),kernel_initializer='he_normal', kernel_regularizer=l2(l=0.01),padding='same'),
        layers.LeakyReLU(alpha=0.2),
        layers.Conv2D(64, (3, 3), kernel_initializer='he_normal', kernel_regularizer=l2(l=0.01),padding='same'),
        layers.Dropout(0.3),
        
        layers.Conv2D(128, (3, 3),strides=(2,2), kernel_initializer='he_normal', kernel_regularizer=l2(l=0.01),padding='same'),
        layers.LeakyReLU(alpha=0.2),
        layers.Conv2D(128, (3, 3),strides=(2,2), kernel_initializer='he_normal', kernel_regularizer=l2(l=0.01),padding='same'),
        layers.LeakyReLU(alpha=0.2),
        layers.Dropout(0.4),
        
        layers.Flatten(),
        layers.Dense(512, kernel_regularizer=l2(l=0.01)),
        layers.LeakyReLU(alpha=0.2),
        layers.Dropout(0.2),
        layers.Dense(512, kernel_regularizer=l2(l=0.01)),
        layers.LeakyReLU(alpha=0.2),
        layers.Dropout(0.2),
        layers.Dense(n_classes)
    ])
    return model

In [25]:
#Define the supervised discriminator. 
#Multiclass classification, so we will use softmax activation.
#To avoid converting our labels to categorical, we will work with sparse categorical crossentropy loss. 

def define_sup_discriminator(disc):
    model=Sequential()
    model.add(disc)
    model.add(layers.Activation('softmax'))
    #Let us use sparse categorical loss so we dont have to convert our Y to categorical
    model.compile(optimizer=Adam(lr=0.0002, beta_1=0.5), 
                  loss="sparse_categorical_crossentropy",metrics=['accuracy'])
    
    return model

In [26]:
# custom activation function for the unsupervised discriminator
#D(x) = Z(x) / (Z(x) + 1) where Z(x) = sum(exp(l(x))). l(x) is the output from sup discr. prior to softmax

def custom_activation(x):
    Z_x = K.sum(K.exp(x), axis=-1, keepdims=True)
    D_x = Z_x /(Z_x+1)
    
    return D_x

#You can also try the built in tensorflow function: tf.reduce_logsumexp(class_logits, 1)

def define_unsup_discriminator(disc):
    model=Sequential()
    model.add(disc)
    model.add(layers.Lambda(custom_activation))
    model.compile(loss='binary_crossentropy', optimizer=Adam(lr=0.0002, beta_1=0.5))
    return model

## ------------------------

### GAN

In [27]:
# define the combined generator and discriminator model, for updating the generator
def define_gan(gen_model, disc_unsup):
    disc_unsup.trainable = False # make unsup. discriminator not trainable
    model = Sequential()
    
    model.add(gen_model)
    
    model.add(disc_unsup)
    
    opt = Adam(lr=0.0002, beta_1=0.5, beta_2=0.444, decay=1.0e-8, amsgrad=True)
    model.compile(loss='binary_crossentropy', optimizer=opt)
    return model


In [28]:
import math
# train the generator and discriminator
def train(gen_model, disc_unsup, disc_sup, gan_model, dataset,unData, latent_dim, n_epochs=20, n_batch=100):

    # select supervised dataset for training.
    #Remember that we are not using all 60k images, just a subset (100 images, 10 per class. )
    X_sup, y_sup = select_supervised_samples(dataset)
    # 	print(X_sup.shape, y_sup.shape)

    bat_per_epo = int(dataset[0].shape[0] / n_batch)

    # iterations
    n_steps = bat_per_epo * n_epochs

    half_batch = int(n_batch / 2)
#     print('n_epochs=%d, n_batch=%d, 1/2=%d, b/e=%d, steps=%d' % (n_epochs, 
#                                                           n_batch, half_batch, 
#                                                           bat_per_epo, n_steps))
    #  enumerate epochs
    for i in range(n_steps):
        # update supervised discriminator (disc_sup) on real samples.
        #Remember that we use real labels to train as this is supervised. 
        #This is the discriminator we really care about at the end.
        #Also, this is a multiclass classifier, not binary. Therefore, our y values 
        #will be the real class labels for MNIST. (NOT 1 or 0 indicating real or fake.)
        [Xsup_real, ysup_real], _ = fetch_sup_samples([X_sup, y_sup], half_batch)

        sup_loss, sup_acc = disc_sup.train_on_batch(Xsup_real, ysup_real)

        # update unsupervised discriminator (disc_unsup) - just like in our regular GAN.
        #Remember that we will not train on labels as this is unsupervised, just binary as in our regular GAN.
        #The y_real below indicates 1s telling the discriminator that these images are real. 
        #do not confuse this with class labels. 
        #We will discard this discriminator at the end. 

        X_real, y_real = fetch_unsup_samples(unData, half_batch) #
        d_loss_real = disc_unsup.train_on_batch(X_real, y_real)

        #Now train on fake. 
        X_fake, y_fake = generate_fake_samples(gen_model, latent_dim, half_batch)
        d_loss_fake = disc_unsup.train_on_batch(X_fake, y_fake)

        # update generator (gen) - like we do in regular GAN.
        #We can discard this model at the end as our primary goal is to train a multiclass classifier (sup. disc.)
        X_gan, y_gan = generate_latent_points(latent_dim, n_batch), ones((n_batch, 1))
        gan_loss = gan_model.train_on_batch(X_gan, y_gan)

        # summarize loss on this batch
        print('>%d, c[%.3f,%.0f], d[%.3f,%.3f], g[%.3f]' % (i+1, sup_loss, sup_acc*100, d_loss_real, d_loss_fake, gan_loss))
        if(math.isnan(gan_loss)):
            break;
        # evaluate the model performance periodically
        if (i+1) % (bat_per_epo * 1) == 0:
            summarize_performance(i, gen_model, disc_sup, latent_dim, dataset)


In [41]:
# report accuracy and save plots & the model periodically.
def summarize_performance(step, gen_model, disc_sup, latent_dim, dataset, n_samples=100):
    # Generate fake images
    X, _ = generate_fake_samples(gen_model, latent_dim, n_samples)

    X = (X + 1) / 2.0 # scale to [0,1] for plotting
    # plot images
    X = (X*255).astype(np.uint8)
    for i in range(12):
        plt.subplot(4, 4, 1 + i)
        plt.axis('off')
        plt.imshow(X[i, :, :, 0])
    # save plot to drive
    filename1 = 'generated_plot_%04d.png' % (step+1)
    plt.savefig('GAN/gen_pic/'+filename1)
    plt.close()

    # evaluate the discriminator
    X, y = dataset
    _, acc = disc_sup.evaluate(X, y, verbose=0)
    print('Discriminator Accuracy: %.3f%%' % (acc * 100))
    # save the generator model
    filename2 = 'gen_model_%04d.h5' % (step+1)
    gen_model.save('GAN/generator/'+filename2)
    
    # save the Discriminator (classifier) model
    filename3 = 'disc_sup_%04d.h5' % (step+1)
    disc_sup.save('GAN/classifier/'+filename3)
    print('>Saved: %s, %s, and %s' % (filename1, filename2, filename3))

## ------------------------

## 4.2 Model training

In [61]:
latent_dim = 100

# create the discriminator models
disc=define_discriminator() #Bare discriminator model... 
disc_sup=define_sup_discriminator(disc) #Supervised discriminator model
disc_unsup=define_unsup_discriminator(disc) #Unsupervised discriminator model. 

gen_model = define_generator(latent_dim) #Generator
gan_model = define_gan(gen_model, disc_unsup) #GAN
dataset = load_main_samples(traindata) #Define the dataset by loading real samples. (This will be a list of 2 numpy arrays, X and y)
extraData = load_extra_samples(trainExtra)
# train the model
# NOTE: 1 epoch = 600 steps in this example. 
with tf.device('/GPU:0'): 
    train(gen_model, disc_unsup, disc_sup, gan_model, dataset,extraData, latent_dim, n_epochs=150, n_batch=32)

Adding images: 100%|█████████████████████████████████████████████████████████████| 7916/7916 [00:00<00:00, 8068.69it/s]



getImage COMPLETED!


Adding images: 100%|█████████████████████████████████████████████████████████████| 7916/7916 [00:00<00:00, 8188.05it/s]



getImage COMPLETED!
>1, c[19.601,19], d[17.998,19.416], g[17.879]
>2, c[19.492,12], d[17.721,19.201], g[17.635]
>3, c[19.302,38], d[17.466,18.999], g[17.399]
>4, c[18.897,25], d[17.252,18.762], g[17.173]
>5, c[18.502,50], d[17.029,18.511], g[16.971]
>6, c[18.213,31], d[16.834,18.256], g[16.772]
>7, c[18.302,25], d[16.622,17.995], g[16.578]
>8, c[17.783,19], d[16.450,17.777], g[16.366]
>9, c[17.619,19], d[16.251,17.611], g[16.152]
>10, c[17.378,38], d[16.018,17.400], g[15.958]
>11, c[17.401,6], d[15.807,17.182], g[15.762]
>12, c[17.223,19], d[15.603,16.972], g[15.564]
>13, c[17.123,25], d[15.385,16.793], g[15.369]
>14, c[16.548,25], d[15.204,16.616], g[15.172]
>15, c[16.767,25], d[15.010,16.436], g[14.978]
>16, c[16.272,19], d[14.811,16.251], g[14.788]
>17, c[16.106,19], d[14.650,16.063], g[14.604]
>18, c[15.776,38], d[14.466,15.863], g[14.428]
>19, c[15.726,19], d[14.280,15.652], g[14.253]
>20, c[15.502,31], d[14.112,15.447], g[14.084]
>21, c[15.575,12], d[13.951,15.245], g[13.914]
>2

>169, c[4.990,19], d[4.039,3.743], g[5.879]
>170, c[4.950,25], d[4.054,3.802], g[5.414]
>171, c[4.955,38], d[3.799,3.849], g[6.092]
>172, c[5.016,19], d[3.682,3.673], g[6.502]
>173, c[4.921,25], d[3.678,3.665], g[6.332]
>174, c[4.762,50], d[3.872,3.785], g[6.434]
>175, c[4.869,25], d[3.786,3.659], g[6.073]
>176, c[4.806,50], d[3.541,3.626], g[6.448]
>177, c[4.793,38], d[3.636,3.572], g[6.070]
>178, c[4.724,44], d[3.573,3.597], g[6.308]
>179, c[4.736,25], d[3.634,3.652], g[6.116]
>180, c[4.670,50], d[3.810,3.648], g[6.038]
>181, c[4.845,19], d[3.419,3.504], g[6.423]
>182, c[4.485,69], d[3.779,3.575], g[5.811]
>183, c[4.537,31], d[3.441,3.436], g[6.928]
>184, c[4.945,25], d[3.387,3.432], g[6.397]
>185, c[4.600,25], d[3.477,3.678], g[6.198]
>186, c[4.488,31], d[3.508,3.408], g[6.463]
>187, c[4.525,25], d[3.352,3.489], g[6.342]
>188, c[4.601,19], d[3.292,3.343], g[7.060]
>189, c[4.734,31], d[3.268,3.623], g[7.666]
>190, c[4.577,19], d[3.253,3.312], g[7.658]
>191, c[4.562,38], d[3.235,3.326

...

>36322, c[1.120,56], d[0.790,0.835], g[0.900]
>36323, c[0.892,56], d[0.838,0.895], g[0.911]
>36324, c[0.790,69], d[0.860,0.798], g[0.915]
>36325, c[1.192,56], d[0.923,0.737], g[0.967]
>36326, c[1.187,56], d[0.858,0.797], g[1.001]
>36327, c[0.849,69], d[0.901,0.800], g[0.938]
>36328, c[0.848,81], d[0.852,0.834], g[0.998]
>36329, c[1.035,50], d[0.889,0.720], g[0.936]
>36330, c[0.840,75], d[0.831,0.869], g[0.965]
>36331, c[0.778,81], d[0.843,0.783], g[1.001]
>36332, c[1.012,81], d[0.761,0.936], g[0.980]
>36333, c[0.996,62], d[0.667,0.861], g[0.924]
>36334, c[1.086,56], d[0.869,0.690], g[0.982]
>36335, c[1.080,75], d[0.978,0.704], g[0.984]
>36336, c[0.974,69], d[0.909,0.699], g[0.952]
>36337, c[0.691,88], d[0.929,0.782], g[0.938]
>36338, c[1.245,62], d[0.751,0.765], g[0.935]
>36339, c[0.865,69], d[0.933,0.796], g[0.893]
>36340, c[1.065,62], d[0.799,0.932], g[0.856]
>36341, c[0.796,81], d[0.723,0.934], g[0.868]
>36342, c[0.698,81], d[0.766,0.956], g[0.922]
>36343, c[0.844,75], d[0.755,0.898

>36503, c[1.062,62], d[0.747,0.835], g[0.897]
>36504, c[0.973,75], d[0.729,0.911], g[0.946]
>36505, c[0.823,81], d[0.882,0.861], g[0.886]
>36506, c[0.769,81], d[0.872,0.789], g[0.901]
>36507, c[0.925,75], d[0.911,0.925], g[0.897]
>36508, c[0.797,88], d[0.840,0.734], g[0.873]
>36509, c[0.895,69], d[0.860,0.822], g[0.913]
>36510, c[1.307,44], d[0.827,0.787], g[0.919]
>36511, c[0.843,69], d[0.816,0.801], g[0.899]
>36512, c[0.661,88], d[0.791,0.848], g[0.901]
>36513, c[0.842,75], d[0.767,0.902], g[0.864]
>36514, c[1.135,69], d[0.842,0.865], g[0.920]
>36515, c[0.763,75], d[0.830,0.821], g[0.924]
>36516, c[1.181,56], d[0.814,0.974], g[0.849]
>36517, c[0.821,69], d[0.796,0.900], g[0.828]
>36518, c[1.368,50], d[0.868,0.875], g[0.984]
>36519, c[0.931,81], d[0.890,0.850], g[0.928]
>36520, c[0.982,69], d[0.868,0.784], g[0.946]
>36521, c[0.974,69], d[0.896,0.736], g[0.909]
>36522, c[0.759,81], d[0.867,0.792], g[0.958]
>36523, c[1.418,31], d[0.826,0.825], g[0.886]
>36524, c[1.031,56], d[0.809,0.859

>36677, c[0.827,81], d[0.860,0.819], g[0.877]
>36678, c[0.738,69], d[0.780,0.970], g[0.857]
>36679, c[0.873,81], d[0.674,0.804], g[0.856]
>36680, c[0.888,81], d[0.813,0.813], g[0.865]
>36681, c[0.785,69], d[0.851,0.936], g[0.827]
>36682, c[0.968,81], d[0.778,0.955], g[0.831]
>36683, c[0.916,62], d[0.776,0.894], g[0.925]
>36684, c[0.775,81], d[0.892,0.875], g[0.923]
>36685, c[1.312,44], d[0.857,0.816], g[0.968]
>36686, c[1.084,44], d[0.961,0.764], g[0.907]
>36687, c[0.723,94], d[0.892,0.909], g[0.906]
>36688, c[0.809,69], d[0.836,0.750], g[0.892]
>36689, c[0.854,69], d[0.794,0.770], g[0.892]
>36690, c[0.824,69], d[0.798,0.891], g[0.885]
>36691, c[0.948,81], d[0.818,0.922], g[0.900]
>36692, c[0.787,69], d[0.765,0.793], g[0.911]
>36693, c[0.844,62], d[0.880,0.833], g[0.875]
>36694, c[0.840,62], d[0.838,0.886], g[0.917]
>36695, c[1.016,62], d[0.828,0.862], g[0.879]
>36696, c[1.254,44], d[0.805,0.949], g[0.869]
>36697, c[0.872,69], d[0.792,0.893], g[0.828]
>36698, c[1.020,75], d[0.814,0.843

>36851, c[0.562,81], d[0.772,0.787], g[0.832]
>36852, c[1.079,50], d[0.816,0.934], g[0.784]
>36853, c[0.966,62], d[0.750,0.957], g[0.857]
>36854, c[0.913,69], d[0.732,0.910], g[0.860]
>36855, c[0.874,75], d[0.862,0.924], g[0.887]
>36856, c[1.009,62], d[0.840,0.780], g[0.943]
>36857, c[1.002,69], d[0.874,0.758], g[0.963]
>36858, c[1.159,62], d[0.763,0.748], g[0.945]
>36859, c[0.781,75], d[0.837,0.750], g[1.046]
>36860, c[1.051,62], d[0.894,0.932], g[0.981]
>36861, c[0.951,69], d[1.017,0.734], g[0.975]
>36862, c[0.807,75], d[0.983,0.686], g[1.049]
>36863, c[0.891,62], d[0.954,0.748], g[1.017]
>36864, c[0.786,81], d[0.824,0.775], g[0.921]
>36865, c[1.153,62], d[0.871,0.852], g[0.872]
>36866, c[1.525,31], d[0.929,0.772], g[0.889]
>36867, c[1.089,56], d[0.801,0.882], g[0.888]
>36868, c[0.973,81], d[0.853,0.841], g[0.892]
>36869, c[0.905,75], d[0.792,0.869], g[0.916]
>36870, c[0.908,69], d[0.857,0.791], g[0.933]
>36871, c[0.844,62], d[0.851,0.781], g[0.838]
>36872, c[1.129,69], d[0.865,0.808

>37031, c[0.773,75], d[0.766,0.793], g[0.862]
>37032, c[0.984,62], d[0.868,0.849], g[0.946]
>37033, c[0.658,81], d[0.871,0.796], g[0.853]
>37034, c[1.008,69], d[0.787,0.954], g[0.865]
>37035, c[1.031,62], d[0.804,0.753], g[0.890]
>37036, c[0.853,56], d[0.852,0.770], g[0.854]
>37037, c[1.141,56], d[0.849,0.842], g[0.849]
>37038, c[0.881,50], d[0.901,0.913], g[0.863]
>37039, c[0.737,75], d[0.789,0.795], g[0.885]
>37040, c[0.919,69], d[0.851,0.901], g[0.865]
>37041, c[1.190,44], d[0.883,0.825], g[0.864]
>37042, c[0.643,88], d[0.854,0.943], g[0.848]
>37043, c[1.030,69], d[0.782,0.861], g[0.835]
>37044, c[0.987,62], d[0.789,0.846], g[0.844]
>37045, c[0.809,62], d[0.882,0.892], g[0.872]
>37046, c[0.883,75], d[0.833,0.816], g[0.830]
>37047, c[0.860,81], d[0.846,0.844], g[0.933]
>37048, c[1.122,62], d[0.896,0.823], g[0.872]
>37049, c[0.964,62], d[0.787,0.783], g[0.859]
>37050, c[1.395,38], d[0.861,0.865], g[0.848]
Discriminator Accuracy: 73.332%
>Saved: generated_plot_37050.png, gen_model_3705

## 4.3 Prediction

In [44]:
from tensorflow.keras.models import load_model
classifier= load_model("GAN\classifier\disc_sup_24700.h5")

ValueError: No model found in config file.

In [62]:
from sklearn.metrics import classification_report

valX = GetImage(validata['ImageName'])
valY = validata['cellType']

valX = valX.astype('float32')
valX = (valX - 127.5) / 127.5  # scale from [0,255] to [-1,1] as we will be using tanh activation.

#
predict=disc_sup.predict(valX)
classes=np.argmax(predict,axis=1)
print(classification_report(valY, classes))

Adding images: 100%|█████████████████████████████████████████████████████████████| 1980/1980 [00:00<00:00, 2278.24it/s]



getImage COMPLETED!
              precision    recall  f1-score   support

           0       0.55      0.78      0.65       378
           1       0.71      0.51      0.59       509
           2       0.87      0.84      0.85       816
           3       0.42      0.47      0.44       277

    accuracy                           0.69      1980
   macro avg       0.64      0.65      0.63      1980
weighted avg       0.71      0.69      0.69      1980



In [63]:
class_names = ["fibroblast","inflammatory","epithelial","others"]
extraX = GetImage(extra_data['ImageName'])
extraY = extra_data['isCancerous']

extraX = extraX.astype('float32')
extraY = (extraY - 127.5) / 127.5  # scale from [0,255] to [-1,1] as we will be using tanh activation.

predict=disc_sup.predict(extraX)
classes=np.argmax(predict,axis=1)
names = list()

for i in range(len(classes)):
    names.append(class_names[np.argmax(predict[i])])
    
submit = pd.DataFrame(columns = ['InstanceID',  'patientID',  'ImageName','cellTypeName','cellType','isCancerous',])
submit['InstanceID'] = extra_data['InstanceID']
submit['patientID'] = extra_data['patientID']
submit['ImageName'] = extra_data['ImageName']
submit['isCancerous'] = extra_data['isCancerous']
submit['cellType'] = classes
submit['cellTypeName'] = names
submit.to_csv("labeled_extra.csv", index = False)

Adding images: 100%|███████████████████████████████████████████████████████████| 10384/10384 [00:04<00:00, 2415.48it/s]



getImage COMPLETED!
